# Library

In [1]:
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [43]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Support Vector Machine
<center><img src="https://kr.mathworks.com/discovery/support-vector-machine/_jcr_content/mainParsys/image.adapt.full.medium.jpg/1718266591845.jpg" alt="Support Vector Machine" width="400"></center>

회귀 분석 기법으로 데이터를 분류하기 위해 최적의 결정 경계를 찾는 데 중점을 둔 알고리즘

<br>

결정 경계(Decision Boundary): 데이터를 분류를 위한 경계. 두 클래스 간의 거리를 최대화하는 평면 또는 초평면(hyperplane)을 찾는 것을 목적으로 함  
서포트 벡터(Support Vectors): 결정 경계를 정의하는 사용하는 포인트 

<br>

SVM은 결정 경계가 각 클래스의 서포트 벡터에서 최대한 멀어지도록 하여 마진(margin)을 최대화하는 것을 목표로 함 <br>
이 과정은 선형 또는 비선형 최적화 문제로 해결 <br>

## Margin

<left><img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*RgFWpCEG5AvnmGF5ESy1Tg.png" alt="Linear" width="700"></left>

결정 경계 결정 시 허용할 수 있는 오차의 정도

<br>

<font style="font-size:20px"> Hard Margin </font>
- 결정 경계: 데이터 포인트가 결정 경계에 대해 오차가 없는 것을 전제. 즉, 모든 데이터 포인트가 결정 경계의 마진 바깥쪽에 위치
- 제약 조건: 데이터 포인트가 마진 내에 위치하는 것을 허용 않음

<br>

<font style="font-size:20px"> Soft Margin </font>
- 결정 경계: 데이터 포인트가 결정 경계의 마진 내에 위치할 수 있도록 허용 <br>
    -> 일부 데이터 포인트는 마진을 침범하거나 심지어 결정 경계의 잘못된 쪽에 위치할 수 있음
- 제약 조건 및 페널티: C라는 하이퍼파라미터를 사용하여 마진 침범을 허용하면서도 이를 최소화. C는 마진 침범의 비용을 조절하는 데 사용
    - C가 크면 마진 침범에 대한 페널티가 커지며, 모델은 훈련 데이터에 더 잘 적합, overfitting 가능성 높아짐
    - C가 작으면 모델은 더 많은 마진 침범을 허용하지만 일반화 능력 향상

<br>

<left><img src="https://velog.velcdn.com/images/cyeongy/post/a985a0b0-66e8-41a5-8a69-42a0c4e1dd2f/image.png" alt="Margin according to C" width="800"></left>

## 장단점

<font style="font-size:20px"> 장점 </font>
- 준수한 성능
- 저차원, 고차원 공간의 적은 데이터에 대해서 일반화 능력이 우수
- noise에 대해 유연한 처리 가능

<br>

<font style="font-size:20px"> 단점 </font>
- 커널함수 선택이 명확하지 않음
- 하이퍼파라미터 튜닝에 성능 의존
- 계산량 부담이 있음
- 비선형 커널의 결정경계 해석이 어려움

## 선형 vs 비선형
<left><img src="https://postfiles.pstatic.net/MjAxNzA3MTRfMjAx/MDAxNTAwMDE4MjQ4Nzcz.X-FIB1JxqpSuEFCh6LGXqA38XCMbyqL7wAxVbYu9rgQg.f-D7aYqoRV3euUacKBF4quD0fgm16wBqtf248W2Qs4og.PNG.tjdudwo93/1.png?type=w3" alt="Linear" width="400"></left>

선형 SVM: 데이터가 선형적으로 구분 가능한 경우 사용. <br>
-> 직선(2D)이나 평면(3D) 등으로 데이터를 완벽하게 나눌 수 있을 때 적용

<br>

|Before|After|
|------|-----|
|<left><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FJbP8O%2FbtqX1piAirw%2FiZXk1KdP4MK6KZPUieGdKK%2Fimg.png" alt="Non Linaer" width="300"></left>|<left><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FclNJlg%2FbtqYbIVGvcr%2F7kK4Ncjvxx9JV2DpDRvjn1%2Fimg.png" alt="Non Linaer" width="300"></left>|

<left><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbknXqQ%2FbtqYbJG7bTI%2F8QoZFpvzAtUUy3yN5uRC9k%2Fimg.png" alt="2D result" width="650"></left>

비선형 SVM: 데이터가 선형적으로 구분되지 않는 경우, 커널 트릭(kernel trick)을 사용하여 데이터의 차원을 증가시켜 비선형 경계 학습. <br>
-> 커널 함수는 원래의 입력 공간을 고차원으로 매핑하여 비선형 문제를 선형 문제로 변환

ex) $ x = \{x_1, x_2\} \rightarrow z = \{x_1^2, \sqrt{2}x_1x_2, x_2^2\} $


### Kernel의 종류

<font style="font-size:20px"> Linear </font>

- $K(x, y) = x^T \cdot y$
- 선형 결정 경계를 만들 때 사용

<br>

<font style="font-size:20px"> Polynomial </font>

<left><img src="https://www.researchgate.net/profile/Asa-Ben-Hur/publication/41896604/figure/fig10/AS:667041182785544@1536046419481/The-effect-of-the-degree-of-a-polynomial-kernel-Higher-degree-polynomial-kernels-allow.png" alt="polynomial kernel" width="700"></left>

- $K(x, y) = (\gamma x^T \cdot y + \theta)^d$
- 입력 데이터를 고차원으로 매핑하며, 비선형 결정 경계를 만들 때 사용
- $\gamma, \theta, d$: hyperparameter

<br>

<font style="font-size:20px"> Radial Basis Function (RBF) </font>

<left><img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*kO_kAQ32-qmT-iljdZdkrQ.png" alt="rbf kernel" width="600"></left>

- $K(x, y) = \exp(-\frac{|x-y|^2}{2\sigma^2})$
- 입력 데이터를 고차원으로 매핑하며, 복잡한 비선형 결정 경계를 만들 때 사용
- $\sigma$: hyperparameter

<br>

<font style="font-size:20px"> Sigmoid </font>

<left><img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_svm_kernels_005.png" alt="kernels" width="500"></left>

- $K(x, y) = \tanh(\gamma x^T \cdot y + \theta)$
- 입력 데이터를 고차원으로 매핑하며, 비선형 결정 경계를 만들 때 사용
- $\gamma, \theta$: hyperparameter

<br>


<left><img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/0*N1x2IH92w_YDeUIj" alt="kernels" width="600" style="background-color:white"></left>


## 사용 방법

```python
```

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> from sklearn.svm import SVC
> 
> # linear
> svm = SVC(kernel='linear').fit(X)
> # polynomial
> svm = SVC(kernel='poly', degree, gamma, coef0).fit(X)
> # RBF
> svm = SVC(kernel='rbf', gamma).fit(X)
> # simoid
> svm = SVC(kernel='sigmoid', gamma, coef0).fit(X)
> 
>
> svm.predict(X) # 학습된 특징을 바탕으로 새로운 데이터에 대해 예측
> ```

- Hyperparameters
    - C: 오차의 허용 정도
    - $\gamma$: 비선형 커널에서 데이터 포인트가 커널의 영향력을 결정하는 데 사용하며 $\gamma$ 가 클수록 데이터 포인트의 영향이 좁아짐
    - $\theta$: 커널의 비선형성과 다항식의 영향을 조절하는 데 사용되며, 값이 커질수록 비선형 항이 더 강조
    - degree: polynoimal kernel의 차원 결정


<font style="font-size:17px"> $\gamma$ </font>

<left><img src="https://velog.velcdn.com/images/cyeongy/post/361a529e-cf86-422c-ae34-03e8cdc67712/image.png" alt="gamma" width="600"></left>

- 곡률의 크기를 결정
- 감마 값이 커질수록 학습 데이터에 있는 모든 변수를 집단에 넣으려고함

<br>

In [4]:
penguin = sns.load_dataset('penguins')

In [5]:
# penguin의 X인자를 적절하게 선택하여 SVM으로 species를 맞추는 모델 구축
penguin = penguin.dropna(subset=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']).drop_duplicates()

In [12]:
tsne = TSNE(n_components=2)
temp = tsne.fit_transform(penguin.iloc[:, 2:-1])
temp = pd.DataFrame(temp, columns=['tsne1', 'tsne2'])
temp = pd.concat([temp, penguin.filter(items=['species'])], axis=1)

In [14]:
px.scatter(
    temp,
    x='tsne1',
    y='tsne2',
    color='species',
)

In [34]:
pca = PCA(n_components=3)
temp = pca.fit_transform(penguin.iloc[:, 2:-1])
temp = pd.DataFrame(temp, columns=['pca1', 'pca2', 'pca3'])
temp = pd.concat([temp, penguin.filter(items=['species'])], axis=1)

px.scatter_3d(
    temp,
    x='pca1',
    y='pca2',
    z='pca3',
    color='species',
)

kernel별 성능

In [6]:
species_to_idx = dict(zip(penguin.species.unique(), range(3)))
penguin.species = penguin.species.apply(lambda x: species_to_idx.get(x))
train, test = train_test_split(penguin, test_size=0.3)

In [29]:
results = {}
for kernel in ('linear', 'poly', 'rbf', 'sigmoid'):
    svm = SVC(kernel=kernel)
    svm.fit(train.iloc[:, 2:-1], train.iloc[:, 0])
    result = svm.predict(test.iloc[:, 2:-1])
    mean = (result == test.iloc[:, 0]).mean()

    results.update({kernel: mean})

In [30]:
results

{'linear': 0.9902912621359223,
 'poly': 0.7766990291262136,
 'rbf': 0.7864077669902912,
 'sigmoid': 0.1650485436893204}

polynomial hyperparameter tuning

In [7]:
# gamma = 0.5
results = {}
for gamma in tqdm(np.arange(0.1, 1.1, 0.1)):
    svm = SVC(kernel='poly', gamma=gamma)
    svm.fit(train.iloc[:, 2:-1], train.iloc[:, 0])
    result = svm.predict(test.iloc[:, 2:-1])
    mean = (result == test.iloc[:, 0]).mean()

    results.update({gamma: mean})
results

100%|██████████| 10/10 [00:02<00:00,  4.90it/s]


{0.1: 0.970873786407767,
 0.2: 0.970873786407767,
 0.30000000000000004: 0.9611650485436893,
 0.4: 0.970873786407767,
 0.5: 0.9902912621359223,
 0.6: 0.9611650485436893,
 0.7000000000000001: 0.9805825242718447,
 0.8: 0.970873786407767,
 0.9: 0.970873786407767,
 1.0: 0.9902912621359223}

In [9]:
results = {}
for degree in range(1, 10):
    svm = SVC(kernel='poly', degree=degree)
    svm.fit(train.iloc[:, 2:-1], train.iloc[:, 0])
    result = svm.predict(test.iloc[:, 2:-1])
    mean = (result == test.iloc[:, 0]).mean()

    results.update({degree: mean})
results

{1: 0.7669902912621359,
 2: 0.7766990291262136,
 3: 0.7669902912621359,
 4: 0.7766990291262136,
 5: 0.7864077669902912,
 6: 0.7864077669902912,
 7: 0.7864077669902912,
 8: 0.7864077669902912,
 9: 0.7864077669902912}

In [10]:
results = {}
for coef in np.arange(1, 10, 0.5):
    svm = SVC(kernel='poly', coef0=coef)
    svm.fit(train.iloc[:, 2:-1], train.iloc[:, 0])
    result = svm.predict(test.iloc[:, 2:-1])
    mean = (result == test.iloc[:, 0]).mean()

    results.update({coef: mean})
results

{1.0: 0.7669902912621359,
 1.5: 0.7864077669902912,
 2.0: 0.7864077669902912,
 2.5: 0.7864077669902912,
 3.0: 0.7864077669902912,
 3.5: 0.7766990291262136,
 4.0: 0.7766990291262136,
 4.5: 0.7766990291262136,
 5.0: 0.7766990291262136,
 5.5: 0.7766990291262136,
 6.0: 0.7766990291262136,
 6.5: 0.7766990291262136,
 7.0: 0.7766990291262136,
 7.5: 0.7766990291262136,
 8.0: 0.7766990291262136,
 8.5: 0.7766990291262136,
 9.0: 0.7766990291262136,
 9.5: 0.7766990291262136}

In [12]:
results = {}
for gamma in tqdm(np.arange(0.1, 1.1, 0.1)):
    for degree in range(1, 10):
        for coef in np.arange(0.1, 10, 0.5):
            svm = SVC(kernel='poly', gamma=gamma, degree=degree, coef0=coef)
            svm.fit(train.iloc[:, 2:-1], train.iloc[:, 0])
            result = svm.predict(test.iloc[:, 2:-1])
            mean = (result == test.iloc[:, 0]).mean()

            results.update({(gamma, degree, coef): mean})
results

  0%|          | 0/10 [00:00<?, ?it/s]

## Support Vector Regression

<left><img src="https://www.saedsayad.com/images/SVR_2.png" alt="svr" width="600"></left>

분류 문제와 비슷하나 marign 내에 최대한 많은 데이터가 들어오도록 학습

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> from sklearn.svm import SVR
> 
> # linear
> svr = SVR(kernel='linear').fit(X)
> # polynomial
> svr = SVR(kernel='poly', degree, gamma, coef0).fit(X)
> # RBF
> svr = SVR(kernel='rbf', gamma).fit(X)
> # simoid
> svr = SVR(kernel='sigmoid', gamma, coef0).fit(X)
> 
>
> svr.predict(X) # 학습된 특징을 바탕으로 새로운 데이터에 대해 예측
> ```

In [6]:
# X: tip을 제외한 나머지 인자
# Y: tip
# X인자를 활용하여 tip을 예측할 수 있는 SVR 모델 구축
# + hyperparameter tuning
tips = sns.load_dataset('tips').dropna().drop_duplicates().reset_index(drop=True)

In [11]:
train, test = train_test_split(tips, test_size=0.3)

min_max_scaler = MinMaxScaler()
train.loc[:, ['tip', 'total_bill', 'size']] = \
    min_max_scaler.fit_transform(train.filter(items=['tip', 'total_bill', 'size']))

test.loc[:, ['tip', 'total_bill', 'size']] = \
    min_max_scaler.transform(test.filter(items=['tip', 'total_bill', 'size']))

In [17]:
min_max_scaler.min_

array([-0.11111111, -0.06430666, -0.2       ])

In [16]:
min_max_scaler.data_range_

array([ 9.  , 47.74,  5.  ])

In [21]:
results = {}
for kernel in ('linear', 'poly', 'rbf', 'sigmoid'):
    svr = SVR(kernel=kernel)
    svr.fit(train.loc[:, ['total_bill', 'size']], train.tip)
    result = svr.predict(test.loc[:, ['total_bill', 'size']])
    original_result = result*min_max_scaler.data_range_[0] + min_max_scaler.data_min_[0]
    original_gt = test.tip*min_max_scaler.data_range_[0] + min_max_scaler.data_min_[0]
    # gt = ground truth = real = true

    mse = np.sqrt(((original_result-original_gt)**2).sum() / len(original_gt))
    results.update({kernel: mse})
results

{'linear': 1.1918929111182683,
 'poly': 1.2178346057369651,
 'rbf': 1.2732210364897167,
 'sigmoid': 62.862465843476976}

In [27]:
# sigmoid kernel의 hyperparameter tuning
results = {}
for gamma in tqdm(np.arange(0.1, 1.1, 0.1)):
    for coef in np.arange(0.1, 5, 0.5):
        svr = SVR(kernel='sigmoid', gamma=gamma, coef0=coef)
        svr.fit(train.loc[:, ['total_bill', 'size']], train.tip)
        result = svr.predict(test.loc[:, ['total_bill', 'size']])
        original_result = result*min_max_scaler.data_range_[0] + min_max_scaler.data_min_[0]
        original_gt = test.tip*min_max_scaler.data_range_[0] + min_max_scaler.data_min_[0]
        # gt = ground truth = real = true

        mse = np.sqrt(((original_result-original_gt)**2).sum() / len(original_gt))
        results.update({(gamma, coef): mse})
results

100%|██████████| 10/10 [00:00<00:00, 38.83it/s]


{(0.1, 0.1): 1.2515476593090613,
 (0.1, 0.6): 1.2901580319809995,
 (0.1, 1.1): 1.3217558646083956,
 (0.1, 1.6): 1.4399367866142359,
 (0.1, 2.1): 1.5456535497445456,
 (0.1, 2.6): 1.595720423852873,
 (0.1, 3.1): 1.6163409380859453,
 (0.1, 3.6): 1.6241737880473366,
 (0.1, 4.1): 1.626983852241852,
 (0.1, 4.6): 1.628021860203,
 (0.2, 0.1): 1.2281323293195334,
 (0.2, 0.6): 1.2392932470683744,
 (0.2, 1.1): 1.2913523367916528,
 (0.2, 1.6): 1.36224487966883,
 (0.2, 2.1): 1.4997697831651902,
 (0.2, 2.6): 1.5731170145376496,
 (0.2, 3.1): 1.607138329595954,
 (0.2, 3.6): 1.6210890010820207,
 (0.2, 4.1): 1.6257735516388487,
 (0.2, 4.6): 1.6275754088221976,
 (0.30000000000000004, 0.1): 1.2062666009141154,
 (0.30000000000000004, 0.6): 1.2428235417628537,
 (0.30000000000000004, 1.1): 1.285933938527969,
 (0.30000000000000004, 1.6): 1.3288195459849894,
 (0.30000000000000004, 2.1): 1.4640929323937102,
 (0.30000000000000004, 2.6): 1.5593781605996804,
 (0.30000000000000004, 3.1): 1.6013803197551952,
 (0.300

In [35]:
min_ = min(results.values())

for key, value in results.items():
    if (value-min_) < 1e-10:
        print(key)

(0.30000000000000004, 0.1)


In [46]:
# SVR 코드로 작성
# data: 건보데이터
data = pd.read_csv(
    './data/국민건강보험공단_건강검진정보_2023.CSV',
    encoding='cp949',
    usecols=['허리둘레', '체중(5kg단위)', 'HDL콜레스테롤', '수축기혈압', '이완기혈압', '혈색소'],
).rename(columns={'체중(5kg단위)': '체중'})

data = data.dropna()
data = data.drop_duplicates()

train, test = train_test_split(data, test_size=0.3)

standard_scaler = StandardScaler()
train.iloc[:] = standard_scaler.fit_transform(train)
test.iloc[:] = standard_scaler.transform(test)

results = {}
for kernel in ('linear', 'poly', 'rbf', 'sigmoid'):
    svr = SVR(kernel=kernel)
    svr.fit(train.drop(columns=['허리둘레']), train.허리둘레)
    predicted = svr.predict(test.drop(columns=['허리둘레']))
    origianl_predicted = predicted*standard_scaler.scale_[1] + standard_scaler.mean_[1]
    origianl_gt = test.허리둘레*standard_scaler.scale_[1] + standard_scaler.mean_[1]
    
    mse = np.sqrt(((origianl_predicted-origianl_gt)**2).sum() / len(origianl_gt))

    results.update({kernel: mse})

# model = sm.OLS.from_formula(
#     'scale(허리둘레) ~ scale(체중) + scale(HDL콜레스테롤) + \
#         scale(수축기혈압) + scale(이완기혈압) + scale(혈색소)',
#     data,
#     ).fit()
# model.summary()

In [ ]:
results